In [1]:
import numpy as np

def initialisation():
    array = []
    p = np.full((8), 1/8)
    while len(array) < 8:
        nb = np.random.choice(8, 1, p=p)[0]
        if nb not in array:
            array.append(nb)
            if len(array) < 0:
                p = np.full((8), 1/(8-len(array)))
                p = [0 if i in array else p[i] for i in range(len(p))]
    return array

In [2]:
def get_penalty(individual):
    pen = np.zeros(8)
    for i in range(8):
        for k in range(i):
            if individual[i]+(i-k) <= 7:
                if k!=i and individual[k] == individual[i]+(i-k):
                    pen[i]+=1
            if individual[i]-(i-k) >= 0:
                if k!=i and individual[k] == individual[i]-(i-k):
                    pen[i]+=1
        for k in range(i, 8):
            if individual[i]+(k-i) <= 7:
                if k!=i and individual[k] == individual[i]+(k-i):
                    pen[i]+=1
            if individual[i]-(k-i) >= 0:
                if k!=i and individual[k] == individual[i]-(k-i):
                    pen[i]+=1

        # for i in range(8):
        #     pen[i]+=(individual == individual[i]).sum()-1
    
    return pen

In [82]:
import tqdm
def eight_queens(num_population, termination_loop):
    min_fitness = []
    population = []
    for i in range(num_population):
        population.append(initialisation())

    penalty = []
    for individual in population:
        penalty.append(get_penalty(individual))

    for _ in range(termination_loop):
        random_5_parents = np.random.randint(0, num_population-1, 5)
        min_2_parent = [np.inf,0]
        min_1_parent = [np.inf,0]
        for i in random_5_parents:
            if np.sum(penalty[i]) < min_1_parent[0]:
                min_2_parent = min_1_parent
                min_1_parent = [np.sum(penalty[i]), i]
            elif np.sum(penalty[i]) < min_2_parent[0]:
                min_2_parent = [np.sum(penalty[i]), i]
        p1, p2 = population[min_1_parent[1]], population[min_2_parent[1]]

        crossover_point = np.random.randint(8)
        children_1 = []
        children_2 = []

        children_1+=p1[:crossover_point]
        children_2+=p2[:crossover_point]

        for i in p2[crossover_point:]+p2[:crossover_point]:
            if i not in children_1:
                children_1.append(i)
                
        for i in p1[crossover_point:]+p1[:crossover_point]:
            if i not in children_2:
                children_2.append(i)

        mutation = np.random.choice(2, p = [0.2, 0.8])
        if mutation==1:
            random_position = np.random.randint(8,size=2)
            temp = children_1[random_position[0]]
            children_1[random_position[0]] = children_1[random_position[1]]
            children_1[random_position[1]] = temp

            random_position = np.random.randint(8,size=2)
            temp = children_2[random_position[0]]
            children_2[random_position[0]] = children_2[random_position[1]]
            children_2[random_position[1]] = temp
        
        population.append(children_1)
        population.append(children_2)

        penalty.append(get_penalty(children_1))
        penalty.append(get_penalty(children_2))

        dict = {}
        for tt in range(len(penalty)):
            dict[tt] = np.sum(penalty[tt])
        sort_penalty = sorted(dict.items(), key=lambda x: x[1], reverse=True)

        min_fitness.append(sort_penalty[-1][1])
        if sort_penalty[-1][1] == 0:
            return population[sort_penalty[-1][0]], min_fitness

        a = np.sort([sort_penalty[0][0], sort_penalty[1][0]])
        for t in range(len(a)-1, -1, -1):
            population.pop(a[t])
            penalty.pop(a[t])

    dict = {}
    for tt in range(len(penalty)):
        dict[tt] = np.sum(penalty[tt])
    sort_penalty = sorted(dict.items(), key=lambda x: x[1], reverse=True)
    
    return population[sort_penalty[-1][0]], min_fitness

    

In [45]:
eight_queens(100, 10000)

  3%|▎         | 272/10000 [00:00<00:06, 1531.97it/s]
